In [5]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [6]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'C:/Users/hp/Downloads/data/model/top_model_vgg/bottleneck_fc_model.h5'
train_data_dir = 'C:/Users/hp/Downloads/data/train'
validation_data_dir = 'C:/Users/hp/Downloads/data/validation'
nb_train_samples = 1200
nb_validation_samples = 198
epochs = 50
batch_size = 16

In [7]:
datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
# save the output as a Numpy array
np.save(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
np.save(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Found 1200 images belonging to 2 classes.
Found 192 images belonging to 2 classes.


In [8]:
train_data = np.load(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_train.npy', 'rb'))
# the features were saved in order, so recreating the labels is easy

train_labels = np.array([0] * 600 + [1] * 600)

validation_data = np.load(open('C:/Users/hp/Downloads/data/model/bottle_neck_features/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * 96 + [1] * 96)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('C:/Users/hp/Downloads/data/model/bottleneck_fc_model.h5')


Train on 1200 samples, validate on 192 samples
Epoch 1/50
1200/1200 [==============================] - 2s 2ms/step - loss: 0.8282 - acc: 0.7275 - val_loss: 0.3038 - val_acc: 0.8750
Epoch 2/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.3679 - acc: 0.8433 - val_loss: 0.2878 - val_acc: 0.8802
Epoch 3/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.3030 - acc: 0.8783 - val_loss: 0.2403 - val_acc: 0.9062
Epoch 4/50
1200/1200 [==============================] - 2s 2ms/step - loss: 0.2748 - acc: 0.8867 - val_loss: 0.2551 - val_acc: 0.8854
Epoch 5/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.2068 - acc: 0.9058 - val_loss: 0.2086 - val_acc: 0.9323
Epoch 6/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.1836 - acc: 0.9267 - val_loss: 0.2587 - val_acc: 0.8906
Epoch 7/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.1543 - acc: 0.9358 - val_loss: 0.3157 - val_acc: 0.9062
Epoch 8/50
1200

In [9]:
# train_data = np.load(open('bottleneck_features_train.npy'))
# train_labels = np.array(
#         [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

# validation_data = np.load(open('bottleneck_features_validation.npy'))
# validation_labels = np.array(
#         [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)

Train on 1200 samples, validate on 192 samples
Epoch 1/50
1200/1200 [==============================] - 2s 2ms/step - loss: 0.8381 - acc: 0.7333 - val_loss: 0.3714 - val_acc: 0.8125
Epoch 2/50
1200/1200 [==============================] - 2s 2ms/step - loss: 0.3734 - acc: 0.8458 - val_loss: 0.2904 - val_acc: 0.8802
Epoch 3/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.3232 - acc: 0.8642 - val_loss: 0.2035 - val_acc: 0.9219
Epoch 4/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.2769 - acc: 0.8850 - val_loss: 0.2503 - val_acc: 0.8906
Epoch 5/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.2093 - acc: 0.9125 - val_loss: 0.9929 - val_acc: 0.7448
Epoch 6/50
1200/1200 [==============================] - 2s 1ms/step - loss: 0.2144 - acc: 0.9150 - val_loss: 0.2747 - val_acc: 0.8906
Epoch 7/50
1200/1200 [==============================] - 2s 2ms/step - loss: 0.1558 - acc: 0.9333 - val_loss: 0.2396 - val_acc: 0.9219
Epoch 8/50
1200